# StackOverflow
---

#### Import Libs

In [ ]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as pyo
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
sns.set_style('darkgrid')
pyo.init_notebook_mode()

#### Bert Setting

In [ ]:
#Setting
MAX_SEQUENCE_LENGTH = 32
BATCHSIZE = 72
HUGGINGFACEMODELPATH = 'bert-base-uncased' #hugging face model
#147396, 0.8=117917
training_sample_count = 117000 # 4860
training_epochs = 3 # 3
running_folds = 2 #2

#### Load Data

In [ ]:
#load questions
df_questions = pd.read_csv('./datasets/202002_questions_cleaned.csv')
df_questions = df_questions.iloc[:, [7,12,13,15,16]]
df_questions['is_closed'] = df_questions['closed_date'].apply(np.isfinite)
print(df_questions.head())
print(df_questions.groupby('is_closed').count())
sns.countplot(df_questions['is_closed'])

In [ ]:
title_data = df_questions[['title','is_closed','closed_reason','creation_date']].copy()
body_data = df_questions[['body','is_closed','closed_reason','creation_date']].copy()
# body_data = body_data.dropna()
# title_data = title_data.dropna()

title_data.title =title_data.title.str.lower()
body_data.body =body_data.body.str.lower()

#Remove Time From Timestamp
title_data.creation_date = pd.to_datetime(title_data.creation_date, unit='s')
body_data.creation_date = pd.to_datetime(body_data.creation_date, unit='s')
print(title_data)
print(body_data)

#### Prepare data for training

In [ ]:
#option1: undersampling
df_class_true = df_questions[df_questions['is_closed'] == True]
df_class_false = df_questions[df_questions['is_closed'] == False]

# df_class_false_under = df_class_false.sample(df_class_true.is_closed.count())
# df_train = pd.concat([df_class_false_under, df_class_true], axis=0)

# df_class_false_under = df_class_false.sample(900)
# df_class_true_under = df_class_true.sample(100)
# df_train = pd.concat([df_class_false_under, df_class_true_under], axis=0)

#option2: whole data with class weight
df_train = df_questions

df_train["is_closed"] = df_train["is_closed"].astype(int)
df_train.count

# df_train, df_test = train_test_split(df_train, test_size=0.2, random_state=42)
df_train.sample(frac=1)
# df_test.sample(frac=1)

#compute class weight
class_weight = {0: df_train.shape[0]/2/df_train[df_train['is_closed'] == False].shape[0], 1: df_train.shape[0]/2/df_train[df_questions['is_closed'] == True].shape[0]}
print(class_weight)
print(class_weight)

#### Data Info

#### Save Data

In [ ]:
def save_dataset(features, targets, feature_name, target_name):
    print('Saving training dataset...')

    np.save('./TrainData/' + feature_name + '.npy', features)
    np.save('./TrainData/' + target_name + '.npy', targets)

    print('Saved parsed dataset')

In [ ]:
# save_dataset(title_data['title'], title_data['is_closed'].map({True:1, 'down':-1}), 'features_title', 'targets_title')
# save_dataset(body_data['body'], body_data['is_closed'].map({'up':1, 'down':-1}), 'features_body', 'targets_body')
save_dataset(title_data['title'], title_data['is_closed'], 'features_title', 'targets_title')
save_dataset(body_data['body'], body_data['is_closed'], 'features_body', 'targets_body')
